In [1]:
import os
import importlib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image as PILImage
from random import uniform

Motor minimalista en Python similar a CROBOTS: versión didáctica con visualización en vivo y carga dinámica de bots.

Ejecutar: python crobots_py_starter.py

Características:
- Arena 2D simple con escenarios pseudoaleatorios (posiciones iniciales del robot + obstáculos)
- API de robot para estudiantes: implementar una clase que herede `RobotController` y anular `step(sensors)`
- Sensores: posición/rumbo propio, radar (objetivo más cercano dentro del alcance), estado de impacto
- Acciones: aceleración (-1..1), giro (-pi..pi), disparo() (un solo proyectil)
- Armas: proyectil simple con velocidad, alcance y colisión
- Bucle de simulación por turnos (dt fijo)
- Visualización con matplotlib (posiciones, rumbos, proyectiles)
- Carga dinámica de bots desde módulos externos de Python para facilitar su uso en el aula

In [2]:
# --- Parámetros generales ---
ROBOT_RADIO = 10
PROYECTIL_RADIO = 2
PROYECTIL_VEL = 100
ROBOT_VEL = 10
ARENA_ANCHO = 400
ARENA_ALTO = 400

In [ ]:
# ============================================================
#  CLASES PRINCIPALES
# ============================================================

class Robot:
    """Representa un robot dentro de la arena."""

    def __init__(self, nombre, clase_control, x, y, arena):
        self.nombre = nombre
        self.control = clase_control(self)
        self.arena = arena
        self.x = x
        self.y = y
        self.angulo = uniform(0, 2 * np.pi)  # Dirección inicial aleatoria
        self.salud = 100
        self.tiempo_disparo = 0

    def mover(self, dt):
        '''
        Mueve el robot en la dirección de su velocidad actual. 
        '''
        dx = np.cos(self.angulo) * ROBOT_VEL * dt 
        dy = np.sin(self.angulo) * ROBOT_VEL * dt
        self.x = np.clip(self.x + dx, ROBOT_RADIO, ARENA_ANCHO - ROBOT_RADIO)
        self.y = np.clip(self.y + dy, ROBOT_RADIO, ARENA_ALTO - ROBOT_RADIO)

    def puede_disparar(self):
        return self.tiempo_disparo <= 0

    def actualizar_cooldown(self, dt):
        self.tiempo_disparo = max(0, self.tiempo_disparo - dt)


class Proyectil:
    """Representa un proyectil en vuelo."""

    def __init__(self, x, y, angulo, propietario):
        self.x = x
        self.y = y
        self.angulo = angulo
        self.propietario = propietario
        self.vivo = True

    def mover(self, dt):
        self.x += np.cos(self.angulo) * PROYECTIL_VEL * dt
        self.y += np.sin(self.angulo) * PROYECTIL_VEL * dt
        if not (0 <= self.x <= ARENA_ANCHO and 0 <= self.y <= ARENA_ALTO):
            self.vivo = False


class Arena:
    """La arena del combate, que gestiona robots y proyectiles."""

    def __init__(self):
        self.robots = []
        self.proyectiles = []
        self.tiempo = 0

    def agregar_robot(self, robot):
        self.robots.append(robot)

    def paso(self, dt):
        """Avanza la simulación un paso."""
        self.tiempo += dt

        # Pedir decisiones a cada robot activo
        for r in self.robots:
            if r.salud > 0:
                r.control.decision()
                r.mover(dt)
                r.actualizar_cooldown(dt)

        # Mover proyectiles
        for p in self.proyectiles:
            p.mover(dt)

        # Colisiones proyectil-robot
        for p in list(self.proyectiles):
            for r in self.robots:
                if r is p.propietario or r.salud <= 0:
                    continue
                if np.hypot(p.x - r.x, p.y - r.y) < ROBOT_RADIO: # Condicion para que impacte
                    r.salud -= 25 # Este daño se podria ajustar  desde el robot
                    p.vivo = False # El proyectil desaparece tras impactar

        # Eliminar proyectiles muertos
        self.proyectiles = [p for p in self.proyectiles if p.vivo]

    def disparar(self, robot):
        """Crea un proyectil si el robot puede disparar."""
        if robot.puede_disparar():
            nuevo = Proyectil(robot.x, robot.y, robot.angulo, robot)
            self.proyectiles.append(nuevo)
            robot.tiempo_disparo = 0.8  # segundos entre disparos, tambien se deberia poden configurar desde el robot

    def render(self, ax):
        """Dibuja el estado actual de la arena. Es medio una cagada pero funciona. Capaz usando skins para los robots, los obstaculos y los disparos"""
        ax.cla()
        ax.set_xlim(0, ARENA_ANCHO)
        ax.set_ylim(0, ARENA_ALTO)
        ax.set_aspect('equal')
        ax.set_title(f"Tiempo: {self.tiempo:.1f}s")

        # Robots
        for r in self.robots:
            color = "blue" if r.salud > 0 else "red"
            ax.add_patch(plt.Circle((r.x, r.y), ROBOT_RADIO, color=color, alpha=0.6))
            ax.arrow(r.x, r.y, np.cos(r.angulo)*15, np.sin(r.angulo)*15,
                     head_width=5, fc='black', ec='black') # la flecha Indica la dirección del robot, el `15 es el largo para que se vea`

        # Proyectiles
        for p in self.proyectiles:
            ax.add_patch(plt.Circle((p.x, p.y), PROYECTIL_RADIO, color="orange"))


In [ ]:
# ============================================================
#  FUNCIONES AUXILIARES
# ============================================================

def cargar_bots_desde_carpeta(carpeta="bots"):
    """Importa todos los bots en la carpeta dada."""
    bots = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".py") and archivo != "__init__.py":
            nombre_modulo = archivo[:-3] # Quitar .py, medio choto pero anda
            modulo = importlib.import_module(f"{carpeta}.{nombre_modulo}")
            if hasattr(modulo, "Bot"):
                bots.append((nombre_modulo, modulo.Bot))
    print(f"Se cargaron {len(bots)} bots desde la carpeta '{carpeta}'.")
    print(bots)
    return bots


def guardar_gif(arena, nombre_archivo="combate.gif", pasos=300, dt=0.1, fps=20):
    """Guarda la simulación completa como un GIF."""
    import matplotlib
    matplotlib.use("Agg")
    import matplotlib.pyplot as plt
    import numpy as np
    from PIL import Image as PILImage

    fig, ax = plt.subplots(figsize=(6, 6))
    frames = []

    for i in range(pasos):
        arena.paso(dt)
        arena.render(ax)
        fig.canvas.draw()

        # Convertir canvas a imagen
        image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        frames.append(PILImage.fromarray(image[..., :3]))

        vivos = [r for r in arena.robots if r.salud > 0]
        if len(vivos) <= 1:
            break

    plt.close(fig)

    frames[0].save(
        nombre_archivo,
        save_all=True,
        append_images=frames[1:],
        duration=int(1000/fps),
        loop=0
    )
    return nombre_archivo


def ejecutar_combate():
    """Carga los bots, ejecuta la simulación y devuelve el ganador y el GIF."""
    arena = Arena()
    bots_disponibles = cargar_bots_desde_carpeta()

    if not bots_disponibles:
        raise RuntimeError("No se encontraron bots en la carpeta 'bots'.")

    # Posiciones iniciales
    for i, (nombre, clase_bot) in enumerate(bots_disponibles):
        x = 50 + i * 100
        y = uniform(50, ARENA_ALTO - 50)
        arena.agregar_robot(Robot(nombre, clase_bot, x, y, arena))

    archivo_gif = guardar_gif(arena, pasos=400, dt=0.1, fps=20)

    # Determinar ganador
    robots_vivos = sorted(arena.robots, key=lambda r: r.salud, reverse=True)
    ganador = robots_vivos[0].nombre if robots_vivos else "Nadie"

    print(f"Ganador: {ganador}")
    print(f"GIF guardado en: {archivo_gif}")
    return ganador, archivo_gif


# if __name__ == "__main__":
#     ejecutar_combate()

In [ ]:
ganador, gif = ejecutar_combate()

In [ ]:
from IPython.display import Image
Image(filename=gif)